# Installation 
___

### Gluon
___

#### For mac:
1. Go to terminal
    * Optional: activate a virtual environment
2. Run these commands
    * pip install gluon
    * pip install mxnet

#### For windows:
1. Go to http://landinghub.visualstudio.com/visual-cpp-build-tools and download and install the C++ compiler.
2. Go to conda prompt
    * Optional: activate a virtual environment
3. Run these commands
    * pip install gluon
    * pip install mxnet
    
___

### Pytorch
___
To use pytorch go to http://pytorch.org and follow the instructions based on your machine.

# Documentation 

---


# Ease of Use
___

## Network

___

#### Gluon

In [3]:
#imports necessary
import mxnet as mx
from mxnet import gluon, autograd, ndarray
import numpy as np
import matplotlib.pyplot as plt

# Initialize the model
net = gluon.nn.Sequential()
    
# Declare hyperparameters
convo1_kernels = 20
convo1_kernel_size = (5,5)
convo2_kernels = 40
convo2_kernel_size = (5,5)
pooling = 2

hidden1_neurons = 20
dropout_rate = 0.3
hidden2_neurons = 15

# Define our network
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=convo1_kernels, kernel_size=convo1_kernel_size, use_bias=True, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=pooling, strides=pooling))
    net.add(gluon.nn.BatchNorm())
    net.add(gluon.nn.Conv2D(channels=convo2_kernels, kernel_size=convo2_kernel_size, use_bias=True, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=pooling, strides=pooling))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(hidden1_neurons, activation="relu", use_bias=True))
    net.add(gluon.nn.Dropout(dropout_rate))
    net.add(gluon.nn.Dense(hidden2_neurons, activation="relu", use_bias=True))
    net.add(gluon.nn.Dense(1, activation="sigmoid", use_bias=True)) # Output layer

### Pytorch
---

In [4]:
#imports used
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable

"""
Class that defines the neural network being used. __init__ defines the layers
in the neural netwrok, such as dense and convolutional layers. The forward function
pushes he input patterns through the network using max pooling for convolution, it
defines the activation function for every layer and returns the output of the network
as x. The function num_flat_features calculates the size of the flat array for the input
of the first dense layer after the last pooling convolutional layer.
"""
class Net(nn.Module):
    def __init__(self, depth, nk, kernel_size, padding, hidden_neurons, nc):
        super(Net, self).__init__()
        # out_channels defines the number of kernels
        self.conv1 = nn.Conv2d(in_channels=depth, out_channels=nk[0], kernel_size=kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(in_channels=nk[0], out_channels=nk[1], kernel_size=kernel_size)
        #self.conv2_drop = nn.Dropout2d()
        # nc is the image size after convolution and pooling
        self.fc1 = nn.Linear(nc * nc * nk[1], hidden_neurons[0])
        self.fc2 = nn.Linear(hidden_neurons[0], hidden_neurons[1])
        # Single value output
        self.fc3 = nn.Linear(hidden_neurons[1], 1)

    def forward(self, x, pooling):
        # Max pooling over a square window with stride of pool size to avoid overlaps
        # Activatin functions are specified in this function even for the convolutional layer
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=pooling, stride=pooling)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=pooling, stride=pooling)
        #x = F.max_pool2d(F.relu(self.conv2_drop(self.conv2(x))), kernel_size=pooling, stride=pooling)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features